# 一. 构建两个场景链

In [30]:

# 构建两个场景的模板
flower_care_template = """
你是一个经验丰富的园丁，擅长解答关于养花育花的问题。
下面是需要你来回答的问题:
{input}
"""
flower_deco_template = """
你是一位网红插花大师，擅长解答关于鲜花装饰的问题。
下面是需要你来回答的问题:
{input}
"""
prompt_infos = [
    {
        "key": "flower_care",
        "description": "适合回答关于鲜花护理的问题",
        "template": flower_care_template,
    },
    {
        "key": "flower_decoration",
        "description": "适合回答关于鲜花装饰的问题",
        "template": flower_deco_template,
    }
]

# 构建目标链 ：构建出两个目标链，分别负责处理不同的问题。
from langchain_openai import OpenAI
llm = OpenAI()
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate
chain_map = {}
# 构建多个基础chain
for info in prompt_infos:
    prompt = PromptTemplate(
        template=info['template'],
        input_variables=["input"]
    )
    print("目标提示:\n", prompt)
    # chain = prompt | llm
    chain = LLMChain(
        llm=llm,
        prompt=prompt,
        verbose=True # 控制程序输出详细程度的常见参数。
    )
    # 放到chain_map中
    chain_map[info["key"]] = chain

目标提示:
 input_variables=['input'] template='\n你是一个经验丰富的园丁，擅长解答关于养花育花的问题。\n下面是需要你来回答的问题:\n{input}\n'
目标提示:
 input_variables=['input'] template='\n你是一位网红插花大师，擅长解答关于鲜花装饰的问题。\n下面是需要你来回答的问题:\n{input}\n'


# 二. 构建路由链：用于选择链

In [31]:
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE as RounterTemplate

# 路由模版
# 之前的prompts构建router_template
# 组合成一个字符串，格式为 "key: description"。
destinations = [f"{p['key']}: {p['description']}" for p in prompt_infos]
router_template = RounterTemplate.format(destinations="\n".join(destinations))
print("路由模板:\n", router_template)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)
print("路由提示:\n", router_prompt)

router_chain = LLMRouterChain.from_llm(
    llm,
    router_prompt,
    verbose=True
)

路由模板:
 Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
flower_care: 适合回答关于鲜花护理的问题
flower_decoration: 适合回答关于鲜花装饰的问题

<< INPUT >>
{input}

<< OUTPUT (m

# 3. 构建默认链

In [ ]:
#
from langchain.chains import ConversationChain
# 如果其他两个链都没有选择时，就走默认链
default_chain = LLMChain(
    llm=llm,
    prompt=router_prompt,
    output_key="text",
    verbose=True
)

# 4. 构建多提示链

In [36]:

# 构建多提示链
from langchain.chains.router import MultiPromptChain

chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=chain_map,  
    default_chain=default_chain
)

In [37]:
# 测试1
print(chain.run("如何为玫瑰浇水？"))
# 测试2              
print(chain.run("如何为婚礼场地装饰花朵？"))
# 测试3         
print(chain.run("如何区分阿豆和罗豆？"))

/Users/lianggao/miniconda3/envs/langchain-demo/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new LLMRouterChain chain...

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:

你是一个经验丰富的园丁，擅长解答关于养花育花的问题。
下面是需要你来回答的问题:
如何为玫瑰浇水？


> Finished chain.

1. 观察土壤湿度：首先要根据玫瑰所在的环境和季节来判断是否需要浇水。一般来说，土壤表面干燥了一两厘米就可以浇水了，但是如果土壤表面还有一些湿润的部分，就不需要浇水。

2. 浇水的时间：最好选择在早晨或者傍晚浇水，避免在白天阳光强烈的时候浇水，这样可以减少水分蒸发的损失。

3. 使用适量的水：浇水时，要注意水的量，不要过少也不要过多。一般来说，每次浇水的量应该是花盆容量的1/3左右，这样可以保证水分充足，但又避免积水导致根部


> Entering new LLMRouterChain chain...

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:

你是一位网红插花大师，擅长解答关于鲜花装饰的问题。
下面是需要你来回答的问题:
如何为婚礼场地装饰花朵？


> Finished chain.

首先，为婚礼场地装饰花朵需要考虑以下几点：
1. 主题概念：根据新人的喜好和婚礼主题，选择相应的花材和色彩，营造出浪漫、温馨或者奢华的氛围。
2. 场地布置：根据场地的大小和布局，确定花艺摆放的位置和方式，可以选择在入口处、舞台、桌上等位置装饰花朵。
3. 花材选择：要考虑到季节、花材的耐久性和价格，可以选择鲜花、干花、人造花等不同材质的花卉搭配使用。
4. 色彩搭配：花朵的颜色可以与新人的服装、场地的装饰和氛围


> Entering new LLMRouterChain chain...

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Given a raw text input to a language model select the model prompt 